In [1]:
%load_ext autoreload
%autoreload 2

from utils.data_loading import load_all_color_images, load_all_depth_images, read_trajectory
import rerun as rr
import numpy as np 

In [3]:
import rerun.blueprint as rrb
def setup_blueprint():
    blueprint = rrb.Blueprint(
        rrb.Horizontal(
                rrb.Vertical(
                    rrb.Spatial3DView(name="3D", origin="world", line_grid=rrb.archetypes.LineGrid3D(plane=rr.components.Plane3D(np.array([0, 1, 0])))),
                ),
                rrb.Vertical(
                    rrb.Spatial2DView(name="RGB", origin="world/camera/image", contents="world/camera/image/rgb"),
                    rrb.Spatial2DView(name="Depth", origin="world/camera/image", contents="world/camera/image/depth"),
                    name="2D",
                ),
            ),
    )
    return blueprint

rr.init("something3", spawn=True, default_blueprint=setup_blueprint())

In [4]:
rgb = load_all_color_images()
depth = load_all_depth_images()
traj = read_trajectory("/Users/korth/Projects/dynamic-scene-graphs/data/living_room_1/livingroom1-traj.txt")

In [ ]:
import rerun as rr

rr.set_time(timeline="world/camera/image", sequence=0)

rr.log("world/camera/image", rr.ViewCoordinates.RDF)

rr.log("world/camera/image", rr.Pinhole(
    resolution=[640, 480],
    principal_point=[320, 240],
    focal_length=500,
))

for i, (d, img) in enumerate(zip(depth, rgb[:100])):
    rr.log("world/camera/image/rgb", rr.Image(img, color_model=rr.ColorModel.RGB))
    rr.log("world/camera/image/depth", rr.DepthImage(d/1000))
    rr.set_time(timeline="world/camera/image", sequence=i)

    rr.log("world/camera", rr.Transform3D(
        rotation=rr.RotationAxisAngle(axis=traj[i].rotation_axis_angle, angle=np.linalg.norm(traj[i].rotation_axis_angle)),
        translation=traj[i].translation
    ))